## **Group Participants**


1. **Acquadro Patrizio**
2. **Drugman Tito Nicola**

## Patient-nurse-room assignment

In this project, you are required to solve a healthcare scheduling problem involving the assignment of patients to rooms and nurses to rooms in a hospital. The goal is to develop an optimization model using mip that minimizes the total delay in patient admissions while adhering to a set of operational and resource constraints. This problem is inspired by the Integrated Healthcare Timetabling Competition 2024 (https://ihtc2024.github.io/).

The hospital operates with a set of patients, rooms, and nurses, each with specific characteristics. Patients are defined by their release date, indicating the earliest day they can be admitted, and a due date, specifying the latest allowable admission day. Patients also have a required length of stay. Rooms are characterized by fixed capacities that determine the maximum number of patients they can accommodate at any given time. Additionally, some patients may have incompatibilities with certain rooms, restricting their assignment.

Nurses have predefined rosters detailing their availability across days. Each occupied room must have one assigned nurse on every day of the scheduling horizon. Nurses can be assigned to a maximum of 3 rooms in a given day.

The problem requires assigning each patient to a room and ensuring that every occupied room has an assigned nurse during each day. These assignments must satisfy several constraints: no room may exceed its capacity, nurse assignments must respect their availability, and patients must only be assigned to compatible rooms. The primary objective is to minimize the total delay in admissions, defined as the sum of the differences between actual admission dates and the release dates of all patients. In what follows we detail the constraints and decisions.


**Constraints:**

- A patient’s length of stay must be respected, meaning they occupy a room for the full duration of their stay.
- A room cannot exceed its capacity on any given day.
- Patients can only be assigned to rooms that they are compatible with.
- Each occupied room must have exactly one assigned nurse per day.
- Nurses can only be assigned to rooms on days when they are available.
- Nurses can be assigned to a maximum of 3 different rooms in the same day.
- Admission days must be within the specified release and due dates.

**Decisions:**

- Assign each patient to a room, ensuring that the same room is assigned for the duration of their stay.
- Determine the admission day for each patient within the allowed range of their release and due dates.
- Assign one nurse to each occupied room on every day of the scheduling period.

The objective is to minimize the total admission delay, defined as the sum of the differences between the admission day and the release date of patients.




# **Big Project Assignment: ANALYSIS**


## 1. Mathematical Model (Plain Markdown)

Below is a plain‐text style model description suitable for a Markdown cell in Google Colab. It does **not** rely on LaTeX syntax, so it should display cleanly without requiring special rendering.

---

### Sets

- **I** = set of patients (indexed by **i**)
- **R** = set of rooms (indexed by **r**)
- **D** = set of days (indexed by **d** or **t**)
- **N** = set of nurses (indexed by **n**)

---

### Parameters

- **Cap[r]**: capacity of room r  
- **L[i]**: length of stay for patient i (number of days)  
- **Rel[i]**: earliest admission day (release date) for patient i  
- **Due[i]**: latest admission day (due date) for patient i  
- **Incomp[i]**: set of rooms that are incompatible with patient i  
- **Shifts[n]**: set of days on which nurse n is available

---

### Decision Variables

1. **x[i, r, d] in {0,1}**  
   - x[i, r, d] = 1 if patient i **starts** in room r on day d,  
   - x[i, r, d] = 0 otherwise.

2. **y[r, d, n] in {0,1}**  
   - y[r, d, n] = 1 if nurse n is assigned to room r on day d,  
   - y[r, d, n] = 0 otherwise.

3. **z[r, t] in {0,1}**  
   - z[r, t] = 1 if room r is occupied by at least one patient on day t,  
   - z[r, t] = 0 if empty.

---

### Objective Function

Minimize total admission delay:

Sum over all i, r, d: (d - Rel[i]) * x[i, r, d]

- If a patient starts on day d, the delay is (d - Rel[i]).
- The goal is to keep patients’ start days as close to their release dates as possible.

---

### Constraints

1. **Unique Admission per Patient**  
   For each patient i:  
   Sum over (r in R, d in D) of x[i, r, d] = 1  
   (Each patient starts exactly once.)

2. **Room Capacity per Day**  
   For each room r, each day t:  
   Sum over (i in I, d in D where d <= t < d + L[i]) of x[i, r, d] <= Cap[r]  
   (No more patients occupy room r on day t than its capacity.)

3. **Room Incompatibility**  
   For each patient i, room r in Incomp[i], day d:  
   x[i, r, d] = 0  
   (Patient i cannot be assigned to any incompatible room.)

4. **Admission Window**  
   For each patient i, room r, day d:  
   If d < Rel[i] or d > Due[i], then x[i, r, d] = 0  
   (Start day must be within [Rel[i], Due[i]].)

5. **Exactly One Nurse per Occupied Room**  
   - Occupied check:
     For each (r, t),  
     Sum of x[i, r, d] over i in I, d in D with d <= t < d + L[i]  
       >= z[r, t]  and  <= Cap[r] * z[r, t]
     (z[r, t] = 1 if at least one occupant, 0 otherwise.)
   - One nurse if occupied:  
     For each (r, t),  
     Sum over n in N of y[r, t, n] = z[r, t]

6. **Nurse Availability**  
   For each (r, d, n):  
   If d not in Shifts[n], then y[r, d, n] = 0  
   (Nurse n can only work on days they are available.)

7. **Max 3 Rooms per Nurse per Day**  
   For each (n, d):  
   Sum over r in R of y[r, d, n] <= 3  
   (A nurse can handle at most 3 rooms on a single day.)

---

## 2. MIP Algorithm (Branch-and-Bound)

Below is a code-like outline for how a typical solver finds an optimal solution:


### Complexity and Exactness

- **Worst-case**: Exponential time (MIP is NP-hard).  
- **Exactness**: If the solver exhaustively explores the branch-and-bound tree, it proves the solution is optimal. If stopped early, you get the best known solution but not necessarily a proof of optimality.


# **Big Project Assignment: SOLUTION**


In [ ]:
# ---------------------------- Section 1: Import Neede Library ----------------------------

# Aggiorniamo i repository e installiamo le librerie di base
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y build-essential libffi-dev -qq > /dev/null 2>&1

# Installiamo cffi alla versione più recente disponibile su PyPI
!pip install --upgrade cffi==1.15.1 -q > /dev/null 2>&1

import importlib
import cffi
importlib.reload(cffi)

# Installiamo mip e requests
!pip install mip -q > /dev/null 2>&1
!pip install requests -q > /dev/null 2>&1

import requests
import pandas as pd
from mip import Model, xsum, minimize, BINARY, OptimizationStatus


# ---------------------------- Section 2: Data Loading and Preprocessing ----------------------------
"""
In this section, we will:

- Load the data from the provided JSON file (GitHub raw URL or local path).
- Extract the sets and parameters:
  - Days (0, 1, 2, ..., up to planning horizon)
  - Rooms and their capacities
  - Patients, their release/due dates, length of stay, incompatible rooms
  - Nurses and their available working shifts
- Store this information in Python data structures for easy reference in our model.
"""

# URL of the JSON file (example for the "big project")
data_url = "https://raw.githubusercontent.com/GiacomoColosio02/Integrated-Healthcare-Timetabling-Competition-2024/refs/heads/main/test02.json"
# Sostituisci con l'URL effettivo che contiene i dati del big project

# Fetch the JSON data
response = requests.get(data_url)
if response.status_code != 200:
    raise ValueError("Failed to download JSON data from the provided URL.")
data = response.json()

# Extract top-level parameters
num_days = data.get("days", None)
if num_days is None:
    raise KeyError("The 'days' key is missing from the JSON data.")

# Extract room data
rooms_data = data.get("rooms", [])
if not rooms_data:
    raise KeyError("No 'rooms' data found in the JSON file.")

# Extract patient data
patients_data = data.get("patients", [])
if not patients_data:
    raise KeyError("No 'patients' data found in the JSON file.")

# Extract nurse data
nurses_data = data.get("nurses", [])
if not nurses_data:
    raise KeyError("No 'nurses' data found in the JSON file.")

# Create sets and parameters
days = range(num_days)
rooms = [room['id'] for room in rooms_data]
room_capacity = {room['id']: room['capacity'] for room in rooms_data}

patients = [p['id'] for p in patients_data]
patient_release = {p['id']: p['release_date'] for p in patients_data}
patient_due = {p['id']: p['due_date'] for p in patients_data}
patient_length_of_stay = {p['id']: p['length_of_stay'] for p in patients_data}
patient_incompatible_rooms = {p['id']: p['incompatible_room_ids'] for p in patients_data}

nurses = [n['id'] for n in nurses_data]
nurse_shifts = {n['id']: [shift['day'] for shift in n['working_shifts']] for n in nurses_data}

# OPTIONAL: Convert to DataFrame for a quick glance (not mandatory)
rooms_df = pd.DataFrame(rooms_data)
patients_df = pd.DataFrame(patients_data)
nurses_df = pd.DataFrame(nurses_data)





# ---------------------------- Section 3: Model Formulation ----------------------------
model = Model("Healthcare_Scheduling_BigProject", solver_name="CBC")

"""
We define a binary decision variable x[i, r, d] = 1
if and only if patient i starts their admission in room r on day d.

Since each patient has a length_of_stay (L), occupying that room from day d to day d + L - 1 (inclusive).
We'll use x[i,r,d] to capture the 'start day' logic. Then we will enforce capacity constraints on the
interval [d, d+L-1].

We also define a binary decision variable y[r, d, n] = 1 if nurse n is assigned to room r on day d, 0 otherwise.
"""

# Decision variables for patient admission (start day)
x = {
    (i, r, d): model.add_var(var_type=BINARY, name=f"x_{i}_{r}_{d}")
    for i in patients
    for r in rooms
    for d in days
}

# Decision variables for nurse assignment
y = {
    (r, d, n): model.add_var(var_type=BINARY, name=f"y_{r}_{d}_{n}")
    for r in rooms
    for d in days
    for n in nurses
}


"""
Objective Function:
Minimize sum of (admission_day(i) - release_date(i)) for all patients i.

If x[i,r,d] = 1 means patient i is admitted on day d.
The cost for that assignment is (d - release_date[i]).

We sum over all i, r, d, multiplying (d - release_date[i]) by x[i,r,d].
"""
admission_delay = xsum(
    (d - patient_release[i]) * x[i, r, d]
    for i in patients
    for r in rooms
    for d in days
    if d >= patient_release[i] and d <= patient_due[i]
)
model.objective = minimize(admission_delay)




# ---------------------------- CONSTRAINTS ----------------------------

"""
1) Each patient is admitted exactly once (to exactly one room, on exactly one start day).

   sum_{r in R, d in D} x[i,r,d] = 1   for all i.
"""
for i in patients:
    model += xsum(x[i, r, d] for r in rooms for d in days) == 1, f"One_Admission_{i}"



"""
2) Respect the capacity of each room on each day.

   For each room r and each day t,
   the total number of patients occupying r on day t cannot exceed room_capacity[r].

   If x[i,r,d] = 1 and the length_of_stay for i is L_i,
   then the patient i occupies room r for days d, d+1, ..., d+L_i-1.

   So we must check for each day t if t is in [d, d+L_i-1].

   For each room r and day t:
       sum_{i in I, d in [0..days]} x[i,r,d] * isOccupied(i, d, t) <= capacity[r]
   where isOccupied(i,d,t) = 1 if t in [d, d+L_i-1], else 0.
"""
for r in rooms:
    for t in days:
        # sum over all patients i and all start days d
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) <= room_capacity[r], f"Capacity_{r}_Day_{t}"



"""
3) Room incompatibility:
   If patient i is incompatible with room r, x[i,r,d] must be 0 for all days d.
"""
for i in patients:
    for r in rooms:
        if r in patient_incompatible_rooms[i]:
            for d in days:
                model += x[i, r, d] == 0, f"Incompatible_{i}_{r}_{d}"



"""
4) Admission day must be within [release_date[i], due_date[i]].
   If day d is outside that interval, x[i, r, d] = 0.
"""
for i in patients:
    for r in rooms:
        for d in days:
            if d < patient_release[i] or d > patient_due[i]:
                model += x[i, r, d] == 0, f"Outside_Window_{i}_{r}_{d}"



"""
5) Nurse assignment:
   Every occupied room on each day must have exactly one nurse assigned (for that day).
   If on day t, there is at least one patient occupying room r,
   sum of y[r,t,n] over nurses n must be 1.

   Actually, we must check the presence of patients.
   If no patient is occupying room r on day t, we do not need any nurse (0).
   If at least one patient is occupying room r on day t, we need exactly 1 nurse.

   Because capacity can be > 1, multiple patients can share the same room,
   but we still require exactly one nurse for that entire room on day t.

   Occupied(r,t) = sum_{i,d : i occupies r on day t} x[i,r,d].
   If Occupied(r,t) >= 1, we need exactly 1 nurse.
   We'll enforce equality:

   sum_{n in N} y[r,t,n] = min(1, Occupied(r,t))

   But note that Occupied(r,t) can be 0, 1, 2,... up to room capacity.
   We want "1 nurse if Occupied(r,t) >= 1, otherwise 0 nurses".
   We can use a trick: because Occupied(r,t) is integer,
   we can add a constraint that sum_n y[r,t,n] >= Occupied(r,t) / M
   with a big M approach OR we can simply force:

   sum_{n} y[r,t,n] = 1 if Occupied(r,t) > 0, else 0.

   One simpler approach:
       sum_{n} y[r,t,n] = 0 if Occupied(r,t) = 0
       sum_{n} y[r,t,n] = 1 if Occupied(r,t) >= 1

   We can do this by bounding:
       sum_{n} y[r,t,n] = min(1, Occupied(r,t))

   We'll handle it piecewise:
   - If Occupied(r,t) = 0, obviously the sum of y[r,t,n] must be 0.
   - If Occupied(r,t) >= 1, sum of y[r,t,n] = 1.

   In MIP, a direct piecewise min(...) is tricky, so we do:

       sum_{n} y[r,t,n] <= Occupied(r,t)
       sum_{n} y[r,t,n] >= Occupied(r,t) / C  (where C = capacity[r] or 1)
       and sum_{n} y[r,t,n] <= 1
       If Occupied(r,t) >= 1, we want exactly 1 nurse.

   A more direct approach: Because capacity can be multiple patients,
   we want 1 nurse if there's ANY occupant. That means:
       sum_n y[r,t,n] <= 1        (can't have more than 1 nurse)
       sum_n y[r,t,n] >= xsum( x[i,r,d] ) / capacity[r], for all i
   But that might produce fractional results in some corner cases.

   Alternatively, we can model a binary variable z[r,t] = 1 if room r is occupied on day t, 0 otherwise,
   then sum_n y[r,t,n] = z[r,t],
   and z[r,t] = 1 if sum_{i,d} x[i,r,d] (for day t) > 0.
   This is a standard 'logical' approach.

   For simplicity, let's define an auxiliary binary variable z[r,t] to capture whether room r is used on day t.
"""

# Auxiliary variable: z[r,t] = 1 if room r is occupied on day t, 0 otherwise.
z = {
    (r, t): model.add_var(var_type=BINARY, name=f"z_{r}_{t}")
    for r in rooms
    for t in days
}

# z[r,t] >= 1 if there's at least one patient occupying room r on day t,
# z[r,t] = 0 otherwise. We link it with x[i,r,d].
for r in rooms:
    for t in days:
        # If the sum of patients occupying r on day t is >= 1, then z[r,t] must be 1
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) <= room_capacity[r] * z[r, t], f"z_UpperBound_{r}_{t}"

        # Also, if there is any occupant, z[r,t] must be at least 1
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) >= z[r, t], f"z_LowerBound_{r}_{t}"

# Then we require: sum_{n} y[r,t,n] = z[r,t],
# i.e. exactly 1 nurse if the room is occupied, 0 if not.
for r in rooms:
    for t in days:
        model += xsum(y[r, t, n] for n in nurses) == z[r, t], f"Exactly_One_Nurse_{r}_{t}"



"""
6) Nurse availability:
   A nurse n can only work on day d if d is in nurse_shifts[n].
   Therefore y[r,d,n] = 0 if day d not in nurse_shifts[n].
"""
for n in nurses:
    for r in rooms:
        for d in days:
            if d not in nurse_shifts[n]:
                model += y[r, d, n] == 0, f"Nurse_Not_Available_{n}_{r}_{d}"



"""
7) Each nurse can be assigned to at most 3 rooms per day.
   sum_{r in R} y[r,d,n] <= 3   for each day d, nurse n.
"""
for n in nurses:
    for d in days:
        model += xsum(y[r, d, n] for r in rooms) <= 3, f"Nurse_Max3Rooms_{n}_{d}"




# ---------------------------- Section 4: Solve & Results ----------------------------
status = model.optimize()

if status == OptimizationStatus.OPTIMAL:
    print("Optimal solution found.")
    print(f"Total admission delay = {model.objective_value}")

    # Retrieve actual assignments
    for i in patients:
        for r in rooms:
            for d in days:
                if x[i, r, d].x >= 0.99:
                    print(f"Patient {i} -> Room {r}, Admission Day {d}. (Stay length = {patient_length_of_stay[i]})")

    print("\nNurse assignments:")
    for r in rooms:
        for d in days:
            for n in nurses:
                if y[r, d, n].x >= 0.99:
                    print(f"Nurse {n} assigned to Room {r} on Day {d}.")

elif status == OptimizationStatus.FEASIBLE:
    print("A feasible solution was found (not guaranteed optimal).")
    print(f"Objective value (admission delay) = {model.objective_value}")
else:
    print("No feasible solution found.")


#Print results in a table
print("SOLUTION:")

table_data = []
for t in days:
    row_data = []
    for r in rooms:
        patients_in_room = []
        for i in patients:
            for d in days:
                if x[i, r, d].x >= 0.99 and (t >= d and t < d + patient_length_of_stay[i]):
                    patients_in_room.append(str(i))
                    break

        nurse_assigned = None
        for n in nurses:
            if y[r, t, n].x >= 0.99:
                nurse_assigned = n
                break


        if patients_in_room:
            cell_content = f"Patient(s): {', '.join(patients_in_room)}; Nurse: {nurse_assigned}"
        else:
            cell_content = "Empty"
        row_data.append(cell_content)
    table_data.append(row_data)

schedule_df = pd.DataFrame(
    table_data,
    index=[f"Day {t}" for t in days],
    columns=[f"Room {r}" for r in rooms]
)

display(schedule_df)



Optimal solution found.
Total admission delay = 0.0
Patient 0 -> Room 3, Admission Day 0. (Stay length = 3)
Patient 1 -> Room 2, Admission Day 0. (Stay length = 2)
Patient 2 -> Room 4, Admission Day 0. (Stay length = 4)
Patient 3 -> Room 2, Admission Day 0. (Stay length = 3)
Patient 4 -> Room 1, Admission Day 1. (Stay length = 2)
Patient 5 -> Room 4, Admission Day 1. (Stay length = 3)
Patient 6 -> Room 1, Admission Day 0. (Stay length = 1)
Patient 7 -> Room 0, Admission Day 2. (Stay length = 2)
Patient 8 -> Room 3, Admission Day 0. (Stay length = 3)
Patient 9 -> Room 2, Admission Day 2. (Stay length = 3)
Patient 10 -> Room 3, Admission Day 3. (Stay length = 2)
Patient 11 -> Room 0, Admission Day 4. (Stay length = 2)
Patient 12 -> Room 0, Admission Day 4. (Stay length = 1)
Patient 13 -> Room 1, Admission Day 1. (Stay length = 2)
Patient 14 -> Room 0, Admission Day 2. (Stay length = 3)
Patient 15 -> Room 1, Admission Day 3. (Stay length = 1)
Patient 16 -> Room 0, Admission Day 0. (Stay l

,Room 0,Room 1,Room 2,Room 3,Room 4
Day 0,"Patient(s): 16, 19; Nurse: 2",Patient(s): 6; Nurse: 0,"Patient(s): 1, 3; Nurse: 2","Patient(s): 0, 8; Nurse: 0",Patient(s): 2; Nurse: 0
Day 1,"Patient(s): 16, 19; Nurse: 0","Patient(s): 4, 13; Nurse: 1","Patient(s): 1, 3; Nurse: 0","Patient(s): 0, 8; Nurse: 1","Patient(s): 2, 5; Nurse: 0"
Day 2,"Patient(s): 7, 14, 19; Nurse: 1","Patient(s): 4, 13; Nurse: 0","Patient(s): 3, 9; Nurse: 0","Patient(s): 0, 8; Nurse: 1","Patient(s): 2, 5; Nurse: 0"
Day 3,"Patient(s): 7, 14; Nurse: 2",Patient(s): 15; Nurse: 0,Patient(s): 9; Nurse: 0,Patient(s): 10; Nurse: 0,"Patient(s): 2, 5; Nurse: 1"
Day 4,"Patient(s): 11, 12, 14; Nurse: 1",Patient(s): 17; Nurse: 1,Patient(s): 9; Nurse: 2,Patient(s): 10; Nurse: 4,Empty
Day 5,"Patient(s): 11, 18; Nurse: 3",Patient(s): 17; Nurse: 3,Empty,Empty,Empty
Day 6,Patient(s): 18; Nurse: 3,Patient(s): 17; Nurse: 4,Empty,Empty,Empty


In [ ]:
# ---------------------------- Section 1: Import Needed Library ----------------------------
# Aggiorniamo i repository e installiamo le librerie di base
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y build-essential libffi-dev -qq > /dev/null 2>&1

# Installiamo cffi alla versione più recente disponibile su PyPI
!pip install --upgrade cffi==1.15.1 -q > /dev/null 2>&1

import importlib
import cffi
importlib.reload(cffi)

# Installiamo mip e requests
!pip install mip -q > /dev/null 2>&1
!pip install requests -q > /dev/null 2>&1

import time   # <-- For measuring computational time
import requests
import pandas as pd
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# ---------------------------- Section 2: Data Loading and Preprocessing ----------------------------
"""
In this section, we will:

- Load the data from the provided JSON file (GitHub raw URL or local path).
- Extract the sets and parameters:
  - Days (0, 1, 2, ..., up to planning horizon)
  - Rooms and their capacities
  - Patients, their release/due dates, length of stay, incompatible rooms
  - Nurses and their available working shifts
- Store this information in Python data structures for easy reference in our model.
"""

# URL of the JSON file
data_url = "https://raw.githubusercontent.com/GiacomoColosio02/Integrated-Healthcare-Timetabling-Competition-2024/refs/heads/main/test02.json"
# Replace with the actual URL containing your project's data if needed

# Fetch the JSON data
response = requests.get(data_url)
if response.status_code != 200:
    raise ValueError("Failed to download JSON data from the provided URL.")
data = response.json()

# Extract top-level parameters
num_days = data.get("days", None)
if num_days is None:
    raise KeyError("The 'days' key is missing from the JSON data.")

# Extract room data
rooms_data = data.get("rooms", [])
if not rooms_data:
    raise KeyError("No 'rooms' data found in the JSON file.")

# Extract patient data
patients_data = data.get("patients", [])
if not patients_data:
    raise KeyError("No 'patients' data found in the JSON file.")

# Extract nurse data
nurses_data = data.get("nurses", [])
if not nurses_data:
    raise KeyError("No 'nurses' data found in the JSON file.")

# Create sets and parameters
days = range(num_days)
rooms = [room['id'] for room in rooms_data]
room_capacity = {room['id']: room['capacity'] for room in rooms_data}

patients = [p['id'] for p in patients_data]
patient_release = {p['id']: p['release_date'] for p in patients_data}
patient_due = {p['id']: p['due_date'] for p in patients_data}
patient_length_of_stay = {p['id']: p['length_of_stay'] for p in patients_data}
patient_incompatible_rooms = {p['id']: p['incompatible_room_ids'] for p in patients_data}

nurses = [n['id'] for n in nurses_data]
nurse_shifts = {
    n['id']: [shift['day'] for shift in n['working_shifts']]
    for n in nurses_data
}

# OPTIONAL: Convert to DataFrame for a quick glance (not mandatory)
rooms_df = pd.DataFrame(rooms_data)
patients_df = pd.DataFrame(patients_data)
nurses_df = pd.DataFrame(nurses_data)


# ---------------------------- Section 3: Model Formulation ----------------------------
model = Model("Healthcare_Scheduling_BigProject", solver_name="CBC")

"""
We define a binary decision variable x[i, r, d] = 1
if and only if patient i starts their admission in room r on day d.

Since each patient has a length_of_stay (L), occupying that room from day d to day d + L - 1 (inclusive).
We'll use x[i,r,d] to capture the 'start day' logic. Then we will enforce capacity constraints on the
interval [d, d+L-1].

We also define a binary decision variable y[r, d, n] = 1 if nurse n is assigned to room r on day d, 0 otherwise.
"""

# Decision variables for patient admission (start day)
x = {
    (i, r, d): model.add_var(var_type=BINARY, name=f"x_{i}_{r}_{d}")
    for i in patients
    for r in rooms
    for d in days
}

# Decision variables for nurse assignment
y = {
    (r, d, n): model.add_var(var_type=BINARY, name=f"y_{r}_{d}_{n}")
    for r in rooms
    for d in days
    for n in nurses
}

"""
Objective Function:
Minimize the sum of (admission_day(i) - release_date(i)) for all patients i.

If x[i,r,d] = 1 it means patient i is admitted on day d.
The 'cost' for that assignment is (d - release_date[i]).

We sum over all i, r, d, multiplying (d - release_date[i]) by x[i,r,d].
"""
admission_delay = xsum(
    (d - patient_release[i]) * x[i, r, d]
    for i in patients
    for r in rooms
    for d in days
    if d >= patient_release[i] and d <= patient_due[i]
)
model.objective = minimize(admission_delay)


# ---------------------------- CONSTRAINTS ----------------------------

# 1) Each patient is admitted exactly once (to exactly one room, on exactly one start day).
for i in patients:
    model += xsum(x[i, r, d] for r in rooms for d in days) == 1, f"One_Admission_{i}"


# 2) Capacity constraints: for each room r and each day t, the total number of patients
# occupying r on day t cannot exceed room_capacity[r].
for r in rooms:
    for t in days:
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) <= room_capacity[r], f"Capacity_{r}_Day_{t}"


# 3) Room incompatibility: if a patient i is incompatible with room r, x[i,r,d] = 0 for all d
for i in patients:
    for r in rooms:
        if r in patient_incompatible_rooms[i]:
            for d in days:
                model += x[i, r, d] == 0, f"Incompatible_{i}_{r}_{d}"


# 4) Admission window: day d must be within [release_date[i], due_date[i]]
for i in patients:
    for r in rooms:
        for d in days:
            if d < patient_release[i] or d > patient_due[i]:
                model += x[i, r, d] == 0, f"Outside_Window_{i}_{r}_{d}"


"""
5) Nurse assignment:
   If a room r is occupied on day t, exactly 1 nurse must be assigned.
   If not occupied, 0 nurses.
   We introduce z[r,t] = 1 if room r is occupied at day t, else 0.
"""
z = {
    (r, t): model.add_var(var_type=BINARY, name=f"z_{r}_{t}")
    for r in rooms
    for t in days
}

# Link z[r,t] to x[i,r,d]
for r in rooms:
    for t in days:
        # If the sum of patients is >= 1, then z[r,t] must be 1
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) <= room_capacity[r] * z[r, t]

        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t < d + patient_length_of_stay[i]
        ) >= z[r, t]

# Exactly one nurse if z[r,t] = 1, else 0
for r in rooms:
    for t in days:
        model += xsum(y[r, t, n] for n in nurses) == z[r, t], f"Exactly_One_Nurse_{r}_{t}"


# 6) Nurse availability: y[r,d,n] = 0 if d not in nurse_shifts[n].
for n in nurses:
    for r in rooms:
        for d in days:
            if d not in nurse_shifts[n]:
                model += y[r, d, n] == 0, f"Nurse_Not_Available_{n}_{r}_{d}"


# 7) Each nurse can be assigned to at most 3 rooms per day
for n in nurses:
    for d in days:
        model += xsum(y[r, d, n] for r in rooms) <= 3, f"Nurse_Max3Rooms_{n}_{d}"


# ---------------------------- Section 4: Solve & Results ----------------------------
# Measure computation time using the time module
start_time = time.time()
status = model.optimize()
end_time = time.time()

# Print solver time
elapsed_seconds = end_time - start_time
print(f"Solve time: {elapsed_seconds:.2f} seconds.")

if status == OptimizationStatus.OPTIMAL:
    print("Optimal solution found.")
    print(f"Total admission delay = {model.objective_value}")

    # Retrieve actual assignments
    for i in patients:
        for r in rooms:
            for d in days:
                if x[i, r, d].x >= 0.99:
                    print(f"Patient {i} -> Room {r}, Admission Day {d}. (Stay length = {patient_length_of_stay[i]})")

    print("\nNurse assignments:")
    for r in rooms:
        for d in days:
            for n in nurses:
                if y[r, d, n].x >= 0.99:
                    print(f"Nurse {n} assigned to Room {r} on Day {d}.")

elif status == OptimizationStatus.FEASIBLE:
    print("A feasible solution was found (not guaranteed optimal).")
    print(f"Objective value (admission delay) = {model.objective_value}")
else:
    print("No feasible solution found.")

print("SOLUTION:")
table_data = []
for t in days:
    row_data = []
    for r in rooms:
        patients_in_room = []
        for i in patients:
            for d in days:
                if x[i, r, d].x >= 0.99 and (t >= d and t < d + patient_length_of_stay[i]):
                    patients_in_room.append(str(i))
                    break

        nurse_assigned = None
        for n in nurses:
            if y[r, t, n].x >= 0.99:
                nurse_assigned = n
                break

        if patients_in_room:
            cell_content = f"Patient(s): {', '.join(patients_in_room)}; Nurse: {nurse_assigned}"
        else:
            cell_content = "Empty"
        row_data.append(cell_content)
    table_data.append(row_data)

schedule_df = pd.DataFrame(
    table_data,
    index=[f"Day {t}" for t in days],
    columns=[f"Room {r}" for r in rooms]
)

display(schedule_df)


Solve time: 0.12 seconds.
Optimal solution found.
Total admission delay = 0.0
Patient 0 -> Room 3, Admission Day 0. (Stay length = 3)
Patient 1 -> Room 2, Admission Day 0. (Stay length = 2)
Patient 2 -> Room 4, Admission Day 0. (Stay length = 4)
Patient 3 -> Room 2, Admission Day 0. (Stay length = 3)
Patient 4 -> Room 1, Admission Day 1. (Stay length = 2)
Patient 5 -> Room 4, Admission Day 1. (Stay length = 3)
Patient 6 -> Room 1, Admission Day 0. (Stay length = 1)
Patient 7 -> Room 0, Admission Day 2. (Stay length = 2)
Patient 8 -> Room 3, Admission Day 0. (Stay length = 3)
Patient 9 -> Room 2, Admission Day 2. (Stay length = 3)
Patient 10 -> Room 3, Admission Day 3. (Stay length = 2)
Patient 11 -> Room 0, Admission Day 4. (Stay length = 2)
Patient 12 -> Room 0, Admission Day 4. (Stay length = 1)
Patient 13 -> Room 1, Admission Day 1. (Stay length = 2)
Patient 14 -> Room 0, Admission Day 2. (Stay length = 3)
Patient 15 -> Room 1, Admission Day 3. (Stay length = 1)
Patient 16 -> Room 0

,Room 0,Room 1,Room 2,Room 3,Room 4
Day 0,"Patient(s): 16, 19; Nurse: 2",Patient(s): 6; Nurse: 0,"Patient(s): 1, 3; Nurse: 2","Patient(s): 0, 8; Nurse: 0",Patient(s): 2; Nurse: 0
Day 1,"Patient(s): 16, 19; Nurse: 0","Patient(s): 4, 13; Nurse: 1","Patient(s): 1, 3; Nurse: 0","Patient(s): 0, 8; Nurse: 1","Patient(s): 2, 5; Nurse: 0"
Day 2,"Patient(s): 7, 14, 19; Nurse: 1","Patient(s): 4, 13; Nurse: 0","Patient(s): 3, 9; Nurse: 0","Patient(s): 0, 8; Nurse: 1","Patient(s): 2, 5; Nurse: 0"
Day 3,"Patient(s): 7, 14; Nurse: 2",Patient(s): 15; Nurse: 0,Patient(s): 9; Nurse: 0,Patient(s): 10; Nurse: 0,"Patient(s): 2, 5; Nurse: 1"
Day 4,"Patient(s): 11, 12, 14; Nurse: 1",Patient(s): 17; Nurse: 1,Patient(s): 9; Nurse: 2,Patient(s): 10; Nurse: 4,Empty
Day 5,"Patient(s): 11, 18; Nurse: 3",Patient(s): 17; Nurse: 3,Empty,Empty,Empty
Day 6,Patient(s): 18; Nurse: 3,Patient(s): 17; Nurse: 4,Empty,Empty,Empty
